## Installing Pytorch

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

## Building Fairseq

In [5]:
!git clone https://github.com/pytorch/fairseq.git

fatal: destination path 'fairseq' already exists and is not an empty directory.


In [0]:
import os
os.chdir("fairseq/")

In [7]:
!pip install -r requirements.txt
%run -i 'setup.py' build develop

    100% |████████████████████████████████| 430kB 4.0MB/s 
    100% |████████████████████████████████| 163kB 31.0MB/s 
  Running setup.py bdist_wheel for pycparser ... - \ | done
  Stored in directory: /root/.cache/pip/wheels/f2/9a/90/de94f8556265ddc9d9c8b271b0f63e57b26fb1d67a45564511
Successfully built pycparser
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.6
creating build/lib.linux-x86_64-3.6/fairseq
copying fairseq/search.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/meters.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/utils.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/sequence_scorer.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/options.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/multiprocessing_pdb.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/tokenizer.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/distributed_utils.py -> build/lib.linux-x86_64-3.6/fairseq
copyi

## Model for Afrikaans
### Data Preprop
Upload the data in the form train.en, train.tn, valid.en, valid.tn, test.en, test.tn.

In [0]:
!pip install sentencepiece

    100% |████████████████████████████████| 1.4MB 4.2MB/s 


In [0]:
import sentencepiece as spm
spm.SentencePieceTrainer.Train('--input=../enaf_parallel.train.en --model_prefix=en --vocab_size=8000 --model_type=bpe')
spm.SentencePieceTrainer.Train('--input=../enaf_parallel.train.af --model_prefix=afr --vocab_size=8000 --model_type=bpe')

In [0]:
import sentencepiece as spm
spm.SentencePieceTrainer.Train('--input=../enaf_parallel.train.en --model_prefix=en --vocab_size=16000 --model_type=bpe')
spm.SentencePieceTrainer.Train('--input=../enaf_parallel.train.af --model_prefix=afr --vocab_size=16000 --model_type=bpe')

True

In [0]:
import sentencepiece as spm
spm.SentencePieceTrainer.Train('--input=../enaf_parallel.train.en --model_prefix=en --vocab_size=32000 --model_type=bpe')
spm.SentencePieceTrainer.Train('--input=../enaf_parallel.train.af --model_prefix=afr --vocab_size=32000 --model_type=bpe')

In [0]:
import sentencepiece as spm
spm.SentencePieceTrainer.Train('--input=../enaf_parallel.train.en --model_prefix=en --vocab_size=40000 --model_type=bpe')
spm.SentencePieceTrainer.Train('--input=../enaf_parallel.train.af --model_prefix=afr --vocab_size=40000 --model_type=bpe')

In [0]:
import codecs

eng = spm.SentencePieceProcessor()
eng.Load("en.model")
afr = spm.SentencePieceProcessor()
afr.Load("afr.model")
eng_file = codecs.open('../test.en','w','utf-8')
afr_file = codecs.open('../test.af','w','utf-8')
en_transcriptions = [line.rstrip('\n') for line in codecs.open("../enaf_parallel.test.en", "r", "utf-8")]
afr_transcriptions = [line.rstrip('\n') for line in codecs.open("../enaf_parallel.test.af", "r", "utf-8")]

## This is being weird with the sizes...

for counter in range(0,3000):
  eng_file.write(u' '.join((eng.EncodeAsPieces(en_transcriptions[counter]))))
  eng_file.write('\n')
  afr_file.write(u' '.join((afr.EncodeAsPieces(afr_transcriptions[counter]))))
  afr_file.write('\n')
  
eng_file = codecs.open('../train.en','w','utf-8')
afr_file = codecs.open('../train.af','w','utf-8')
en_transcriptions = [line.rstrip('\n') for line in codecs.open("../enaf_parallel.train.en", "r", "utf-8")]
afr_transcriptions = [line.rstrip('\n') for line in codecs.open("../enaf_parallel.train.af", "r", "utf-8")]

for counter in range(0,37219):
  eng_file.write(u' '.join((eng.EncodeAsPieces(en_transcriptions[counter]))))
  eng_file.write('\n')
  afr_file.write(u' '.join((afr.EncodeAsPieces(afr_transcriptions[counter]))))
  afr_file.write('\n')
  
eng_file = codecs.open('../valid.en','w','utf-8')
afr_file = codecs.open('../valid.af','w','utf-8')
en_transcriptions = [line.rstrip('\n') for line in codecs.open("../enaf_parallel.dev.en", "r", "utf-8")]
afr_transcriptions = [line.rstrip('\n') for line in codecs.open("../enaf_parallel.dev.af", "r", "utf-8")]

for counter in range(0,12953):
  eng_file.write(u' '.join((eng.EncodeAsPieces(en_transcriptions[counter]))))
  eng_file.write('\n')
  afr_file.write(u' '.join((afr.EncodeAsPieces(afr_transcriptions[counter]))))
  afr_file.write('\n')

In [0]:
en_transcriptions = [line.rstrip('\n') for line in codecs.open("../valid.en", "r", "utf-8")]
print(len(en_transcriptions))
eng_file = codecs.open('../valid.en','w','utf-8')
for counter in range(0,12939):
  eng_file.write(u' '.join(en_transcriptions[counter]))
  eng_file.write('\n')

12934


IndexError: ignored

### Subword

In [8]:
!git clone https://github.com/rsennrich/subword-nmt

Cloning into 'subword-nmt'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 481 (delta 3), reused 8 (delta 3), pack-reused 471
Receiving objects: 100% (481/481), 206.42 KiB | 385.00 KiB/s, done.
Resolving deltas: 100% (282/282), done.


In [0]:
os.chdir('../')

In [11]:
# Learn a vocabulary using 40,000 merge operations
!fairseq/subword-nmt/learn_bpe.py -s 40000 <enaf_parallel.train.en> en_train.bpe
!fairseq/subword-nmt/learn_bpe.py -s 40000 <enaf_parallel.train.af> af_train.bpe
# !fairseq/subword-nmt/learn_bpe.py -s 16000 <enaf_parallel.dev.en> en_valid.bpe
# !fairseq/subword-nmt/learn_bpe.py -s 16000 <enaf_parallel.dev.af> af_valid.bpe
# !fairseq/subword-nmt/learn_bpe.py -s 16000 <enaf_parallel.test.en> en_test.bpe
# !fairseq/subword-nmt/learn_bpe.py -s 16000 <enaf_parallel.test.af> af_test.bpe

# Apply the vocabulary to the training file
!fairseq/subword-nmt/apply_bpe.py -c en_train.bpe < enaf_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c af_train.bpe < enaf_parallel.train.af > train.af
!fairseq/subword-nmt/apply_bpe.py -c en_train.bpe < enaf_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c af_train.bpe < enaf_parallel.dev.af > valid.af
!fairseq/subword-nmt/apply_bpe.py -c en_train.bpe < enaf_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c af_train.bpe < enaf_parallel.test.af > test.af

fairseq/subword-nmt/learn_bpe.py:267: DeprecationWarning: this script's location has moved to /content/fairseq/subword-nmt/subword_nmt. This symbolic link will be removed in a future version. Please point to the new location, or install the package and use the command 'subword-nmt'
  DeprecationWarning
no pair has frequency >= 2. Stopping
fairseq/subword-nmt/learn_bpe.py:267: DeprecationWarning: this script's location has moved to /content/fairseq/subword-nmt/subword_nmt. This symbolic link will be removed in a future version. Please point to the new location, or install the package and use the command 'subword-nmt'
  DeprecationWarning
no pair has frequency >= 2. Stopping
fairseq/subword-nmt/apply_bpe.py:334: DeprecationWarning: this script's location has moved to /content/fairseq/subword-nmt/subword_nmt. This symbolic link will be removed in a future version. Please point to the new location, or install the package and use the command 'subword-nmt'
  DeprecationWarning
fairseq/subwor

In [0]:
os.chdir('fairseq/')

In [13]:
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang af --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/enaf

Namespace(alignfile=None, destdir='data-bin/enaf', joined_dictionary=False, nwordssrc=-1, nwordstgt=-1, only_source=False, output_format='binary', padding_factor=8, source_lang='en', srcdict=None, target_lang='af', testpref='../test', tgtdict=None, thresholdsrc=0, thresholdtgt=0, trainpref='../train', validpref='../valid', workers=1)
| [en] Dictionary: 27231 types
| [en] ../train.en: 37219 sents, 527972 tokens, 0.0% replaced by <unk>
| [en] Dictionary: 27231 types
| [en] ../valid.en: 12953 sents, 266909 tokens, 0.486% replaced by <unk>
| [en] Dictionary: 27231 types
| [en] ../test.en: 3000 sents, 51271 tokens, 0.497% replaced by <unk>
| [af] Dictionary: 32319 types
| [af] ../train.af: 37219 sents, 520989 tokens, 0.0% replaced by <unk>
| [af] Dictionary: 32319 types
| [af] ../valid.af: 12953 sents, 277739 tokens, 1.05% replaced by <unk>
| [af] Dictionary: 32319 types
| [af] ../test.af: 3000 sents, 52496 tokens, 0.798% replaced by <unk>
| Wrote preprocessed data to data-bin/enaf


### Train Model

In [14]:
!mkdir -p chckpoint/fconv
!python train.py data-bin/enaf \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir ckpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/enaf --path ckpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang af

Namespace(arch='fconv', bucket_cap_mb=150, clip_norm=0.1, criterion='cross_entropy', data=['data-bin/enaf'], ddp_backend='c10d', decoder_attention='True', decoder_embed_dim=512, decoder_embed_path=None, decoder_layers='[(512, 3)] * 20', decoder_out_embed_dim=256, device_id=0, distributed_backend='nccl', distributed_init_method=None, distributed_port=-1, distributed_rank=0, distributed_world_size=1, dropout=0.2, encoder_embed_dim=512, encoder_embed_path=None, encoder_layers='[(512, 3)] * 20', fix_batches_to_gpus=False, fp16=False, fp16_init_scale=128, fp16_scale_window=None, keep_interval_updates=-1, left_pad_source='True', left_pad_target='False', log_format=None, log_interval=1000, lr=[0.25], lr_scheduler='reduce_lr_on_plateau', lr_shrink=0.1, max_epoch=0, max_sentences=None, max_sentences_valid=None, max_source_positions=1024, max_target_positions=1024, max_tokens=4000, max_update=0, min_loss_scale=0.0001, min_lr=1e-05, momentum=0.99, no_epoch_checkpoints=False, no_progress_bar=False

### Testing
Generate translations from the test data. Calculate the Bleu score.

In [0]:
# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/enaf --path ckpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang af

Namespace(beam=5, cpu=False, data=['data-bin/enaf'], diverse_beam_groups=1, diverse_beam_strength=0.5, fp16=False, fp16_init_scale=128, fp16_scale_window=None, gen_subset='test', left_pad_source='True', left_pad_target='False', lenpen=1, log_format=None, log_interval=1000, max_len_a=0, max_len_b=200, max_sentences=128, max_source_positions=1024, max_target_positions=1024, max_tokens=None, min_len=1, model_overrides='{}', nbest=1, no_beamable_mm=False, no_early_stop=False, no_progress_bar=False, num_shards=1, path='ckpoint/fconv/checkpoint_best.pt', prefix_size=0, print_alignment=False, quiet=False, raw_text=False, remove_bpe='@@ ', replace_unk=None, sampling=False, sampling_temperature=1, sampling_topk=-1, score_reference=False, seed=1, shard_id=0, skip_invalid_size_inputs_valid_test=False, source_lang='en', target_lang='af', task='translation', unkpen=0, unnormalized=False, upsample_primary=1)
| [en] dictionary: 27232 types
| [af] dictionary: 28288 types
| data-bin/enaf test 3000 exam

## Results

*   40k: 15.78
*   32k: 17.49
*   24k: 18.96
*   16k: 18.07
*     8k: 20.04

